# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8638cf1f70>
├── 'a' --> tensor([[ 1.2689, -0.0169,  1.5876],
│                   [-0.3900,  0.5284,  0.4369]])
└── 'x' --> <FastTreeValue 0x7f8638cf1ee0>
    └── 'c' --> tensor([[ 0.4005,  0.9805,  0.6165,  1.5842],
                        [-2.4962,  0.0117, -0.0704,  0.1576],
                        [-0.4448,  1.8329,  0.5763, -0.7583]])

In [4]:
t.a

tensor([[ 1.2689, -0.0169,  1.5876],
        [-0.3900,  0.5284,  0.4369]])

In [5]:
%timeit t.a

63.8 ns ± 0.0401 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8638cf1f70>
├── 'a' --> tensor([[-0.1975, -0.0848,  1.0023],
│                   [-1.3156, -0.6354,  0.1769]])
└── 'x' --> <FastTreeValue 0x7f8638cf1ee0>
    └── 'c' --> tensor([[ 0.4005,  0.9805,  0.6165,  1.5842],
                        [-2.4962,  0.0117, -0.0704,  0.1576],
                        [-0.4448,  1.8329,  0.5763, -0.7583]])

In [7]:
%timeit t.a = new_value

70.6 ns ± 0.0397 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.2689, -0.0169,  1.5876],
               [-0.3900,  0.5284,  0.4369]]),
    x: Batch(
           c: tensor([[ 0.4005,  0.9805,  0.6165,  1.5842],
                      [-2.4962,  0.0117, -0.0704,  0.1576],
                      [-0.4448,  1.8329,  0.5763, -0.7583]]),
       ),
)

In [10]:
b.a

tensor([[ 1.2689, -0.0169,  1.5876],
        [-0.3900,  0.5284,  0.4369]])

In [11]:
%timeit b.a

65.4 ns ± 0.0353 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.4843,  0.7201, -1.0297],
               [ 0.7754, -0.5462,  0.1462]]),
    x: Batch(
           c: tensor([[ 0.4005,  0.9805,  0.6165,  1.5842],
                      [-2.4962,  0.0117, -0.0704,  0.1576],
                      [-0.4448,  1.8329,  0.5763, -0.7583]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.0854 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

902 ns ± 2.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 32 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 262 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 300 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8638cf14c0>
├── 'a' --> tensor([[[ 1.2689, -0.0169,  1.5876],
│                    [-0.3900,  0.5284,  0.4369]],
│           
│                   [[ 1.2689, -0.0169,  1.5876],
│                    [-0.3900,  0.5284,  0.4369]],
│           
│                   [[ 1.2689, -0.0169,  1.5876],
│                    [-0.3900,  0.5284,  0.4369]],
│           
│                   [[ 1.2689, -0.0169,  1.5876],
│                    [-0.3900,  0.5284,  0.4369]],
│           
│                   [[ 1.2689, -0.0169,  1.5876],
│                    [-0.3900,  0.5284,  0.4369]],
│           
│                   [[ 1.2689, -0.0169,  1.5876],
│                    [-0.3900,  0.5284,  0.4369]],
│           
│                   [[ 1.2689, -0.0169,  1.5876],
│                    [-0.3900,  0.5284,  0.4369]],
│           
│                   [[ 1.2689, -0.0169,  1.5876],
│                    [-0.3900,  0.5284,  0.4369]]])
└── 'x' --> <FastTreeValue 0x7f8638cf1eb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.8 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f863cd7a280>
├── 'a' --> tensor([[ 1.2689, -0.0169,  1.5876],
│                   [-0.3900,  0.5284,  0.4369],
│                   [ 1.2689, -0.0169,  1.5876],
│                   [-0.3900,  0.5284,  0.4369],
│                   [ 1.2689, -0.0169,  1.5876],
│                   [-0.3900,  0.5284,  0.4369],
│                   [ 1.2689, -0.0169,  1.5876],
│                   [-0.3900,  0.5284,  0.4369],
│                   [ 1.2689, -0.0169,  1.5876],
│                   [-0.3900,  0.5284,  0.4369],
│                   [ 1.2689, -0.0169,  1.5876],
│                   [-0.3900,  0.5284,  0.4369],
│                   [ 1.2689, -0.0169,  1.5876],
│                   [-0.3900,  0.5284,  0.4369],
│                   [ 1.2689, -0.0169,  1.5876],
│                   [-0.3900,  0.5284,  0.4369]])
└── 'x' --> <FastTreeValue 0x7f85752c6f70>
    └── 'c' --> tensor([[ 0.4005,  0.9805,  0.6165,  1.5842],
                        [-2.4962,  0.0117, -0.0704,  0.1576],
                 

In [23]:
%timeit t_cat(trees)

32.3 µs ± 702 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67.6 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.2689, -0.0169,  1.5876],
                [-0.3900,  0.5284,  0.4369]],
       
               [[ 1.2689, -0.0169,  1.5876],
                [-0.3900,  0.5284,  0.4369]],
       
               [[ 1.2689, -0.0169,  1.5876],
                [-0.3900,  0.5284,  0.4369]],
       
               [[ 1.2689, -0.0169,  1.5876],
                [-0.3900,  0.5284,  0.4369]],
       
               [[ 1.2689, -0.0169,  1.5876],
                [-0.3900,  0.5284,  0.4369]],
       
               [[ 1.2689, -0.0169,  1.5876],
                [-0.3900,  0.5284,  0.4369]],
       
               [[ 1.2689, -0.0169,  1.5876],
                [-0.3900,  0.5284,  0.4369]],
       
               [[ 1.2689, -0.0169,  1.5876],
                [-0.3900,  0.5284,  0.4369]]]),
    x: Batch(
           c: tensor([[[ 0.4005,  0.9805,  0.6165,  1.5842],
                       [-2.4962,  0.0117, -0.0704,  0.1576],
                       [-0.4448,  1.8329,  0.5763, -0.7583]],
         

In [26]:
%timeit Batch.stack(batches)

80.9 µs ± 747 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.2689, -0.0169,  1.5876],
               [-0.3900,  0.5284,  0.4369],
               [ 1.2689, -0.0169,  1.5876],
               [-0.3900,  0.5284,  0.4369],
               [ 1.2689, -0.0169,  1.5876],
               [-0.3900,  0.5284,  0.4369],
               [ 1.2689, -0.0169,  1.5876],
               [-0.3900,  0.5284,  0.4369],
               [ 1.2689, -0.0169,  1.5876],
               [-0.3900,  0.5284,  0.4369],
               [ 1.2689, -0.0169,  1.5876],
               [-0.3900,  0.5284,  0.4369],
               [ 1.2689, -0.0169,  1.5876],
               [-0.3900,  0.5284,  0.4369],
               [ 1.2689, -0.0169,  1.5876],
               [-0.3900,  0.5284,  0.4369]]),
    x: Batch(
           c: tensor([[ 0.4005,  0.9805,  0.6165,  1.5842],
                      [-2.4962,  0.0117, -0.0704,  0.1576],
                      [-0.4448,  1.8329,  0.5763, -0.7583],
                      [ 0.4005,  0.9805,  0.6165,  1.5842],
                      [-2.4962,  

In [28]:
%timeit Batch.cat(batches)

150 µs ± 699 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

345 µs ± 554 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
